#### Implementing dropout regularization

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer, Dropout
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils
import keras
import imageio 
from PIL import Image 

In [ ]:
# Reading the data
train = pd.read_csv('agedetectiontrain/train.csv')

# Image resizing of test data into single numpy array
temp = []
for img_name in train.ID:
    img_path = os.path.join('agedetectiontrain/Train', img_name)
    img = imageio.imread(img_path)
    img = np.array(Image.fromarray(img).resize((32, 32))).astype('float32')    
    temp.append(img)

train_x = np.stack(temp)

# Normalizing the images
train_x = train_x / 255.

# Encoding the categorical variable to numeric
lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.to_categorical(train_y)

In [ ]:
# Specifying all the parameters we will be using in our network
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3

epochs = 100
batch_size = 512

In [ ]:
model = Sequential([
          InputLayer(input_shape=input_num_units),
          Flatten(),
        # Dropout layer can also be added between input layer and first hidden layer.
        # keras.layers.Dropout(0.30),
        
        # According to dropout paper, the maximum norm of the weights should not exceed a value of 3. 
        # Hence, defining constraint maxnorm(3).
          Dense(units=hidden_num_units, activation='relu', kernel_constraint=keras.constraints.maxnorm(3)),
          Dropout(0.30), # Dropout layer with 30% of input units of last layer to drop
          Dense(units=output_num_units, activation='softmax'),
        ])
# Defining parameters like optmizer, loss function and evaluating metric
model.compile(loss='categorical_crossentropy', # 
        optimizer=keras.optimizers.Adam(), # Learning rate and momentum can be passed inside optimizer
        metrics=['accuracy'])

cb = keras.callbacks.TensorBoard(log_dir='dropout', write_graph=False) 
        
history = model.fit(train_x, train_y, epochs=epochs, validation_split=0.2, callbacks=[cb])